# Cluster Analysis - The shape-based approach


In this jupyter notebook, I am going to be building off of the Getting Started notebook by utlizing the following specifications that were derived in the notebook: 

1. dive threshold (meters): 30
2. minimum dive length (number of measurements in the dive): 200
3. number of principle components to take from the count-based representation: 10


First, we are going to make a few crucial imports that will load all tag datasets, and get the dives that we want from them. 

In [1]:
import numpy as np
import utils
import clusterUtils
import tagStruct
import importlib
import matplotlib.pyplot as plt

In this next cell, we are going to derive representations of the dives with the following specifications:
1. pure counts
2. take first 10 P.C.'s

In [2]:
importlib.reload(clusterUtils)
tags = utils.loadDatasets(threshold = 30 , truncationLength = 200)
pureCounts10PCS = []
for i in range(0, len(tags)):
    pureCounts10PCS.append(utils.createRepresentation(tags[i], numComponents = 10, probability = False))

data/5111027PAM110P0574TS.csv
['time', 'depth', 'temperature', 'light level']
found:  time
data/5111033PAM110P0587-Archive.csv
['time', 'depth', 'temperature', 'light level']
found:  time
data/5111034PAM110P0588-Archive.csv
['time', 'depth', 'temperature', 'light level']
found:  time
data/5111045PAM110P0590-Archive.csv
['time', 'depth', 'temperature', 'light level']
found:  time
data/5112030PAM110P0416-Archive.csv
['time', 'depth', 'temperature', 'light level']
found:  time
data/5112039PAM111P0762-Archive.csv
['time', 'depth', 'temperature', 'light level']
found:  time
data/5112041PAM111P0763-Archive.csv
['time', 'depth', 'temperature', 'light level']
found:  time


In [ ]:
import warnings
warnings.filterwarnings("ignore")
allScores = [np.zeros((1, 30-2)) for i in range(0, len(pureCounts10PCS))]
for i in range(0, 50):
    siloScores = [clusterUtils.SC(pureCounts10PCS[i], "cosine", 30) for i in range(0, len(pureCounts10PCS))]
    for j in range(0, len(siloScores)):
        asNp = np.asarray(siloScores[j])
        allScores[j] = allScores[j] + asNp
allScores = [allScores[i]/(50) for i in range(0, len(allScores))]

In [ ]:
for i in range(0, len(allScores)):
    siloScore = allScores[i][0]
    plt.plot(range(1, len(siloScore)+1), siloScore)
    plt.title("The maximum silhouette score was achieved at " + str(np.argmax(np.asarray(siloScore))+1) + " clusters.")
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette Score")
    plt.show()

Quickly, let's look at how the number of principle components that we take from the dive representations affects the number of clusters that maximizes the silhouette score

In [ ]:
pureCounts15PCS = []
for i in range(0, len(tags)):
    pureCounts15PCS.append(utils.createRepresentation(tags[i], numComponents = 20, probability = False))
    
import warnings
warnings.filterwarnings("ignore")
allScores15 = [np.zeros((1, 30-2)) for i in range(0, len(pureCounts15PCS))]
for i in range(0, 50):
    siloScores15 = [clusterUtils.SC(pureCounts15PCS[i], "cosine", 30) for i in range(0, len(pureCounts15PCS))]
    for j in range(0, len(siloScores15)):
        asNp = np.asarray(siloScores15[j])
        allScores15[j] = allScores15[j] + asNp
allScores15 = [allScores15[i]/(50) for i in range(0, len(allScores15))]


for i in range(0, len(allScores)):
    siloScore = allScores15[i][0]
    print(np.argmax(np.asarray(siloScore))+1)
    plt.plot(siloScore)
    plt.show()

Alright, after averaging the silhouette scores after 30 runs, it looks like there is a consistent cluster number that kmediods is settling on: 5-7 clusters! To analyze the clusters, we are going to do the following: 


1. Look at examples from each of the clusters. 
2. Map where the clusters are. 


In [ ]:
optimalClusterNums = [5,5,7,6,7,7,7]
import scipy.spatial.distance
importlib.reload(clusterUtils)
clusterAssignments = [clusterUtils.kMedoids(scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(pureCounts10PCS[i], metric = 'cosine')), optimalClusterNums[i])[1] for i in range(0, len(optimalClusterNums))]
for i in range(0, len(clusterAssignments)):
    listOfClustersExamples = clusterUtils.collectExampleFromClusterAssignments(clusterAssignments[i][1], tags[i])
    clusterUtils.plotExamples(listOfClustersExamples, maxNumToPrint = 10)
    print("______________________________________________________NEW TAG______________________________________________________")

In [ ]:
for i in range(0, len(pureCounts10PCS)):
    print(len(pureCounts10PCS[i]))

In [ ]:
clusterAssignmnets[0]

In [ ]:
importlib.reload(tagStruct)
importlib.reload(clusterUtils)


listyList = list()
for i in range(0, len(tags)):
    labelsVec = np.zeros((1, len(tags[i].dives)))
    print(labelsVec.shape)
    for k in range(0, (len(clusterAssignments[i])-1)):
        divesInCluster = clusterAssignments[k]
        for diveNummy in range(0, len(divesInCluster)):
            xc = divesInCluster[diveNummy]
            print(xc)
            if xc < len(tags[i].dives):
                labelsVec[0][xc] = k
    listyList.append(labelsVec)
    
    
    print("_________________")


In [ ]:
import sklearn.cluster
optimalClusterNums = [5,5,7,6,7,7,7]
for i in range(0, len(pureCounts10PCS)): ##each one of these things is a tag
    k = optimalClusterNums[i]
    kmeans =  sklearn.cluster.KMeans(n_clusters=k).fit(pureCounts10PCS[i])
    labels = kmeans.labels_
    f=open(tags[i].name + 'km.p',"wb")
    pickle.dump(labels, f)
    f.close()


In [ ]:
importlib.reload(tagStruct)
importlib.reload(clusterUtils)
for tag in tags: 
    clusterUtils.getDiveLocations(tag)

In [ ]:
probDissims = []
for i in range(0, len(tags)):
    probDissims.append(utils.createRepresentation(tags[i], numComponents = 0, probability = True))

In [ ]:
siloScores2 = [clusterUtils.SC(probDissims[i], "Jensen-Shannon", 30) for i in range(0, len(probDissims))]

In [ ]:
for i in range(0, len(siloScores)):
    siloScore = siloScores2[i]
    plt.plot(siloScore)
    plt.show()

In [ ]:
importlib.reload(clusterUtils)
BIC_AIC_tuples = [clusterUtils.GausMM(pureCounts10PCS[i], 30) for i in range(0, len(pureCounts10PCS))]

In [ ]:
for i in range(0, len(BIC_AIC_tuples)):
    aic, bic = BIC_AIC_tuples[i]
    averageAIC = np.sum(aic, axis=0)/aic.shape[0]
    averageBIC = np.sum(bic, axis=0)/bic.shape[0]
    plt.plot(averageAIC)
    plt.plot(averageBIC)
    print(np.argmax(averageAIC)+1)
    plt.show()
    
    

Now I am going to look at how spectral clustering works for the data set. 

In [ ]:
importlib.reload(clusterUtils)
import sklearn.metrics as metrics
import sklearn.cluster
for i in range(0, len(pureCounts10PCS)): ##each one of these things is a tag
    distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(pureCounts10PCS[i], metric = 'cosine'))
    simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
    summed = np.zeros((1, 13))
    for s in range(0, 50):
        tempList= list()
        for k in range(2, 15):
            spec =  sklearn.cluster.SpectralClustering(n_clusters=k,
             assign_labels="kmeans",
             random_state=0,
            affinity = 'precomputed').fit(simMatrix)
            silScore = metrics.silhouette_score(distanceMat, spec.labels_, metric="precomputed", sampleSize = None)
            tempList.append(silScore)
        summed = summed + np.asarray(tempList)
    summed[0] = summed[0] /50
    plt.plot(range(1, len(summed[0])+1), summed[0], label = "Spectral Clustering")
    siloScoreKMed = allScores[i][0][0:len(summed[0])]
    plt.plot(range(1, len(summed[0])+1), siloScoreKMed, label = "K-Medoids")
    plt.title("The maximum silhouette score was achieved at " + str(np.argmax(np.asarray(summed[0]))+1) + " clusters for spectral clustering.")
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette Score")
    plt.legend(loc='upper left')
    plt.show()
    print("----------")

So according to silhouette score, it looks like spectral clustering is giving us a superior clustering than k-mediods. Let's look at some example dives in each cluster assigned via silohouette score. 

In [ ]:
importlib.reload(clusterUtils)
import pickle
import sklearn.metrics as metrics
import sklearn.cluster
numClustersSpectClustering = [7,6,9,6,9,6,7]
for i in range(0, len(pureCounts10PCS)): ##each one of these things is a tag
    distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(pureCounts10PCS[i], metric = 'cosine'))
    simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
    spec =  sklearn.cluster.SpectralClustering(n_clusters=numClustersSpectClustering[i],
             assign_labels="kmeans",
             random_state=0,
            affinity = 'precomputed').fit(simMatrix)
    labels = spec.labels_
    f=open(tags[i].name + 'Spectral.p',"wb")
    pickle.dump(labels, f)
    f.close()
    reorderedLabels = clusterUtils.reorderInverse(labels)
    listOfClustersExamples = clusterUtils.collectExampleFromClusterAssignments(reorderedLabels, tags[i])
    clusterUtils.plotExamples(listOfClustersExamples, maxNumToPrint = 10)
    print("______________________________________________________NEW TAG______________________________________________________")
    

Let's look at how varying the principle components changes the silhoutte score. 

In [ ]:
importlib.reload(clusterUtils)
import sklearn.metrics as metrics
import sklearn.cluster
dimsToTry = [5,15, 20, 25, 50]
for comp in dimsToTry:
    pureCountsXPCS = []
    for i in range(0, len(tags)):
        pureCountsXPCS.append(utils.createRepresentation(tags[i], numComponents = comp, probability = False))
        for i in range(0, len(pureCountsXPCS)): ##each one of these things is a tag
            distanceMat = scipy.spatial.distance.squareform(
                    scipy.spatial.distance.pdist(pureCountsXPCS[i], metric = 'cosine'))
            simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
            summed = np.zeros((1, 13))
            for s in range(0, 10):
                tempList= list()
                for k in range(2, 15):
                    spec =  sklearn.cluster.SpectralClustering(n_clusters=k,
                     assign_labels="kmeans",
                     random_state=0,
                    affinity = 'precomputed').fit(simMatrix)
                    silScore = metrics.silhouette_score(distanceMat, spec.labels_, metric="precomputed", sampleSize = None)
                    tempList.append(silScore)
                summed = summed + np.asarray(tempList)
            summed[0] = summed[0] /10
            plt.plot(range(1, len(summed[0])+1), summed[0], label = "Spectral Clustering")
            siloScoreKMed = allScores[i][0][0:len(summed[0])]
            plt.plot(range(1, len(summed[0])+1), siloScoreKMed, label = "K-Medoids")
            plt.title("The maximum silhouette score was achieved at " + str(np.argmax(np.asarray(summed[0]))+1) + " clusters for spectral clustering.")
            plt.xlabel("Number of clusters")
            plt.ylabel("Silhouette Score")
            plt.legend(loc='upper left')
            plt.show()
            print("----------")


Here we are going to use tSNE to see how well this technique can learn the distribution of pairwise distance that we create with the counts at depth and dive velocity approach. Here let's look at the distribution of the 2-d embeddings from tSNE. ***NOTE that from the original count matrices, I'm taking many more principle components. *** 


To run the cells below, first run the first two code blocks at the top of the notebook.  

In [ ]:
from sklearn.manifold import TSNE
import scipy.spatial.distance
from mpl_toolkits.mplot3d import Axes3D



pureCounts10PCS = []
for i in range(0, 1):
    pureCounts10PCS.append(utils.createRepresentation(tags[i], numComponents = 50, probability = False))
                                                      

        
#Adam, the matrix at pureCounts10PCS[0] is like shape (811, numComponents). Whatever the dimension of your embeddings, this is where
#you should just plug in the embeddings, and the code below will plot out how the reps look for tSNE. Right now I'm just looking
#the reps of a single tag, but you coud just do an extra for-loop over the pureCounts10PCS vector. 


        
perps = [2, 5, 10, 20, 50, 70, 100, 150]
distanceMat  = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(pureCounts10PCS[0], metric = 'cosine'))
for i in range(0, len(perps)):
    tsne_2 = TSNE(n_components=2, verbose=1, perplexity=perps[i], n_iter=700, metric = "precomputed")
    tsne_results = tsne_2.fit_transform(distanceMat)
    plt.scatter(x = tsne_results[:, 0],y = tsne_results[:, 1])
    plt.title("t-SNE embeddings with perplexity " + str(perps[i]))
    plt.show()

So we want this stuff to look clusterable, which means some clouds of any shape in distinct region. As you can see, there are some embddings that look promising. Now let's look at the 3-d embeddings. 

In [ ]:
from sklearn.manifold import TSNE
import scipy.spatial.distance
from mpl_toolkits.mplot3d import Axes3D



pureCounts10PCS = []
for i in range(0, len(tags)):
    pureCounts10PCS.append(utils.createRepresentation(tags[i], numComponents = 50, probability = True))  

tsne = list()
for i in range(0, len(tags)):
    distanceMat  = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(pureCounts10PCS[i], metric = 'euclidean'))
    tsne_3 = TSNE(n_components=3, verbose=1, perplexity=20, n_iter=700, metric = "precomputed")
    tsne.append(tsne_3.fit_transform(distanceMat))



In [ ]:
print(tsne_results.shape)
tsne = [tsne_results]

In [ ]:
importlib.reload(clusterUtils)
import sklearn.metrics as metrics
import sklearn.cluster
for i in range(0, len(tsne)): ##each one of these things is a tag
    distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(tsne[i], metric = 'euclidean'))
    simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
    summed = np.zeros((1, 50))
    for s in range(0, 5):
        tempList= list()
        for k in range(2, 52):
            spec =  sklearn.cluster.SpectralClustering(n_clusters=k,
             assign_labels="kmeans",
             random_state=0,
            affinity = 'precomputed').fit(simMatrix)
            silScore = metrics.silhouette_score(distanceMat, spec.labels_, metric="precomputed", sampleSize = None)
            tempList.append(silScore)
        summed = summed + np.asarray(tempList)
    summed[0] = summed[0] /5
    plt.plot(range(1, len(summed[0])+1), summed[0], label = "Spectral Clustering")
    #siloScoreKMed = allScores[i][0][0:len(summed[0])]
    #plt.plot(range(1, len(summed[0])+1), siloScoreKMed, label = "K-Medoids")
    plt.title("The maximum silhouette score was achieved at " + str(np.argmax(np.asarray(summed[0]))+1) + " clusters for spectral clustering.")
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette Score")
    plt.legend(loc='upper left')
    plt.show()
    print("----------")

In [ ]:
distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(tsne[i], metric = 'euclidean'))
simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
summed = np.zeros((1, 13))

In [ ]:
importlib.reload(clusterUtils)
import pickle
import sklearn.metrics as metrics
import sklearn.cluster
from collections import Counter
occs = list()
for i in range(0, len(tsne)):
    distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(tsne[i], metric = 'euclidean'))
    simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
    summed = np.zeros((1, 13))
    spec =  sklearn.cluster.SpectralClustering(n_clusters=15,
                 assign_labels="kmeans",
                 random_state=0,
                affinity = 'precomputed').fit(simMatrix)
    labels = spec.labels_
    reorderedLabels = clusterUtils.reorderInverse(labels)
    f = open(tags[i].name + 'tSNESmoothed.p',"wb")
    pickle.dump(labels, f)
    f.close()
    print(Counter(labels))
    coccurences = np.zeros((15, 15))
    for window in range(1, 2):
        coccurencesTemp = np.zeros((15, 15))
        for k in range(0, len(labels) - window):
            coccurencesTemp[labels[k],labels[k+window]] += 1/window 
        coccurences += coccurencesTemp
    coccurences = coccurences 
    occs.append(coccurences)
    labelsSplit = np.array_split(labels, 10)
    #for i in range(0, len(labelsSplit)):
    #    plt.plot(labelsSplit[i])
    #    plt.show()
        
    #listOfClustersExamples = clusterUtils.collectExampleFromClusterAssignments(reorderedLabels, tags[i])
    #clusterUtils.plotExamples(listOfClustersExamples, maxNumToPrint = 10)
    print("______________________________________________________NEW TAG______________________________________________________")

In [ ]:
for i in range(0, len(occs)):
    #occs[i] = occs[i] + occs[i].T
    plt.imshow(occs[i], cmap = "Blues")
    plt.colorbar()
    plt.title(tags[i].name)
    plt.show()
    
    

In [ ]:
f=open(tags[0].name + 'tSNE.p',"wb")
pickle.dump(labels, f)
f.close()

In [ ]:
fo

In [ ]:
summed

In [ ]:
arma = pickle.load(open('5111027PAM110P0574TS_arma.p','rb'))

In [ ]:
distARMA = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(arma , metric = 'cosine'))
plt.imshow(distARMA, cmap = "hot")
plt.show()

In [ ]:
perps = [5,10,20,40,60,80,100,200]
for i in range(0, len(perps)):
    tsne_2 = TSNE(n_components=3, verbose=1, perplexity=perps[i], n_iter=700, metric = "precomputed")
    tsne_results = tsne_2.fit_transform(distARMA )
    plt.scatter(x = tsne_results[:, 0],y = tsne_results[:, 1])
    plt.show()
    
    
tsne_2 = TSNE(n_components=2, verbose=1, perplexity=20, n_iter=700, metric = "precomputed")
tsne_results = tsne_2.fit_transform(distARMA )

In [ ]:
tsne_2 = TSNE(n_components=2, verbose=1, perplexity=20, n_iter=700, metric = "precomputed")
tsne_results = tsne_2.fit_transform(distARMA)
spec =  sklearn.cluster.SpectralClustering(n_clusters=3,
             assign_labels="kmeans",
             random_state=0,
            affinity = 'nearest_neighbors').fit(tsne_results)
labels = spec.labels_ 
print(len(labels))
reorderedLabels = clusterUtils.reorderInverse(labels)
listOfClustersExamples = clusterUtils.collectExampleFromClusterAssignments(reorderedLabels, tags[0])
clusterUtils.plotExamples(listOfClustersExamples, maxNumToPrint = 10)


In [ ]:
armaList = [arma]
for i in range(0, len(armaList)): ##each one of these things is a tag
    distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(armaList[i], metric = 'euclidean'))
    simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
    summed = np.zeros((1, 50))
    for s in range(0, 5):
        tempList= list()
        for k in range(2, 52):
            print("here")
            spec =  sklearn.cluster.SpectralClustering(n_clusters=k,
             assign_labels="kmeans",
             random_state=0,
            affinity = 'precomputed').fit(simMatrix)
            silScore = metrics.silhouette_score(distanceMat, spec.labels_, metric="precomputed", sampleSize = None)
            tempList.append(silScore)
        summed = summed + np.asarray(tempList)
    summed[0] = summed[0] /5
    plt.plot(range(1, len(summed[0])+1), summed[0], label = "Spectral Clustering")
    #siloScoreKMed = allScores[i][0][0:len(summed[0])]
    #plt.plot(range(1, len(summed[0])+1), siloScoreKMed, label = "K-Medoids")
    plt.title("The maximum silhouette score was achieved at " + str(np.argmax(np.asarray(summed[0]))+1) + " clusters for spectral clustering.")
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette Score")
    plt.legend(loc='upper left')
    plt.show()
    print("----------")

Next, as a final step, we are going to just look at some simple representations of the dives: max depth, time as certain depths, and the average depth

In [ ]:
importlib.reload(utils)

In [ ]:
for i in range(0, len(tags)):
    utils.createSimpleRepresentations(tags[i])

In [ ]:
from sklearn.decomposition import PCA
tempSimpleReps = tags[0].simpleReps
normalized = (tempSimpleReps - tempSimpleReps.mean(axis=0)) / (tempSimpleReps.std(axis=0) + .00001)
pca = PCA(n_components=20)
pca.fit(normalized)
myData = pca.fit_transform(normalized)

In [ ]:
perps = [2, 5, 10, 20, 50]
distanceMat  = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(myData, metric = 'euclidean'))
for i in range(0, len(perps)):
    tsne_2 = TSNE(n_components=2, verbose=1, perplexity=perps[i], n_iter=700, metric = "precomputed")
    tsne_results = tsne_2.fit_transform(distanceMat)
    plt.scatter(x = tsne_results[:, 0],y = tsne_results[:, 1])
    plt.title("t-SNE embeddings with perplexity = " + str(10))
    plt.show()

In [ ]:
tsne_simple = tsne_2 = TSNE(n_components=2, verbose=1, perplexity=10, n_iter=700, metric = "precomputed")
tsne_results_simple = tsne_2.fit_transform(distanceMat)
print(tsne_results.shape)





In [ ]:
armaList = [tsne_results_simple]
for i in range(0, len(armaList)): ##each one of these things is a tag
    distanceMat = scipy.spatial.distance.squareform(
            scipy.spatial.distance.pdist(armaList[i], metric = 'euclidean'))
    simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
    summed = np.zeros((1, 50))
    for s in range(0, 5):
        tempList= list()
        for k in range(2, 52):
            print("here")
            spec =  sklearn.cluster.SpectralClustering(n_clusters=k,
             assign_labels="kmeans",
             random_state=0,
            affinity = 'precomputed').fit(simMatrix)
            silScore = metrics.silhouette_score(distanceMat, spec.labels_, metric="precomputed", sampleSize = None)
            tempList.append(silScore)
        summed = summed + np.asarray(tempList)
    summed[0] = summed[0] /5
    plt.plot(range(1, len(summed[0])+1), summed[0], label = "Spectral Clustering")
    #siloScoreKMed = allScores[i][0][0:len(summed[0])]
    #plt.plot(range(1, len(summed[0])+1), siloScoreKMed, label = "K-Medoids")
    plt.title("The maximum silhouette score was achieved at " + str(np.argmax(np.asarray(summed[0]))+1) + " clusters for spectral clustering.")
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette Score")
    plt.legend(loc='upper left')
    plt.show()
    print("----------")

In [ ]:
simMatrix = clusterUtils.simmMatrix(distanceMat, 1.0)
spec =  sklearn.cluster.SpectralClustering(n_clusters=10,
             assign_labels="kmeans",
             random_state=0,
            affinity = 'precomputed').fit(simMatrix)
labels = spec.labels_

In [ ]:
reorderedLabels = clusterUtils.reorderInverse(labels)
listOfClustersExamples = clusterUtils.collectExampleFromClusterAssignments(reorderedLabels, tags[0])
clusterUtils.plotExamples(listOfClustersExamples, maxNumToPrint = 20)

In [ ]:
plt.scatter(x = simpleSimpleReps[:, 0],y = simpleSimpleReps[:, 2])
plt.show()


In [3]:
importlib.reload(tagStruct)
for tag in tags:
    tag.getThermocline()

(627,)
(627,)


ValueError: Lengths must match to compare